In [1]:
import pandas as pd
import os

In [2]:
# define the base path to thumb drive (input path)
base_path = '/Volumes/Untitled/MeterDataTest'

#check if base path exists
if not os.path.exists(base_path):
    print(f"Error: Path {base_path} does not exist")
    exit()

In [3]:
# list to store all dataframes
all_data = []

# dataframe for all data
combined_data = pd.DataFrame()

In [12]:
# iterate through the subfolders in the MeterDataTest folder
for subfolder in os.listdir(base_path):
    subfolder.replace(" ", "_")
    # create path for each subfolder
    folder_path = os.path.join(base_path, subfolder)

    # get the name of the meter from the subfolder name, make lowercase
    meter_name = subfolder.lower().replace(" ", "_") 

    # list of csv file paths in subfolder
    # addition with the and not is to make sure to ignore the hidden ._ files
    csv_paths = [os.path.join(folder_path, f) 
                 for f in os.listdir(folder_path) 
                 if f.endswith('.csv')
                 and not f.startswith("._")
                 and not f.startswith(".")]

    # convert each csv to a df, fix columns and add df to df list
    for csv in csv_paths:
        df = pd.read_csv(csv, encoding="utf-8")

        df.columns = df.columns.str.strip().str.replace(" ", "_")
        df.insert(0, 'meter_name', meter_name)

        all_data.append(df)

In [13]:
# check list of dataframes, none with 'Unnamed: 0' column
for i, df in enumerate(all_data):
    print(f"DataFrame {i} columns: {df.columns.tolist()}")
    if df.empty:
        print(df)

DataFrame 0 columns: ['meter_name', 'Total_Watt_Hour', '3_phase_watt_total', 'Datetime']
DataFrame 1 columns: ['meter_name', 'Total_Watt_Hour', '3_phase_watt_total', 'Datetime']
DataFrame 2 columns: ['meter_name', 'Total_Watt_Hour', '3_phase_watt_total', 'Datetime']
DataFrame 3 columns: ['meter_name', 'Total_Watt_Hour', '3_phase_watt_total', 'Datetime']
DataFrame 4 columns: ['meter_name', 'Total_Watt_Hour', '3_phase_watt_total', 'Datetime']
DataFrame 5 columns: ['meter_name', 'Total_Watt_Hour', '3_phase_watt_total', 'Datetime']
DataFrame 6 columns: ['meter_name', 'Total_Watt_Hour', '3_phase_watt_total', 'Datetime']
DataFrame 7 columns: ['meter_name', 'Total_Watt_Hour', '3_phase_watt_total', 'Datetime']
DataFrame 8 columns: ['meter_name', 'Total_Watt_Hour', '3_phase_watt_total', 'Datetime']
DataFrame 9 columns: ['meter_name', 'Total_Watt_Hour', '3_phase_watt_total', 'Datetime']
DataFrame 10 columns: ['meter_name', 'Total_Watt_Hour', '3_phase_watt_total', 'Datetime']
DataFrame 11 columns

In [14]:
# combine all dataframes in list to one dataframe
combined_data = pd.concat(all_data, ignore_index=True)

In [15]:
# check dataframe
print(combined_data.columns)
print(combined_data.head(5))

         meter_name  Total_Watt_Hour  3_phase_watt_total             Datetime  \
0  admin_serv_1_mtr        1381508.0        27995.804688  2025-07-23 09:40:50   
1  admin_serv_1_mtr        1381508.0        36522.058594  2025-07-23 09:41:54   
2  admin_serv_1_mtr        1381509.0        27541.597656  2025-07-23 09:42:59   
3  admin_serv_1_mtr        1381510.0        27613.425781  2025-07-23 09:44:03   
4  admin_serv_1_mtr        1381510.0        27681.099609  2025-07-23 09:45:08   

   3_Phase_Positive_Real_Energy_Used  3_phase_real_power  
0                                NaN                 NaN  
1                                NaN                 NaN  
2                                NaN                 NaN  
3                                NaN                 NaN  
4                                NaN                 NaN  


In [10]:
# convert dataframe to csv
combined_data.to_csv('step1.csv', index=False)

In [82]:
# function to combine all the csvs in the subfolder add the meter_name collumn
def combine_csvs(folder_path, meter_name):
    data = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            csv_path = os.path.join(folder_path, file_name)

            # encoding for mac
            df = pd.read_csv(csv_path, encoding="utf-8")
            
            df.insert(loc=0, column='meter_name', value=meter_name)
            data.append(df)
    return pd.concat(data, ignore_index=True) if data else pd.DataFrame()

In [83]:
# iterate through the subfolders in the MeterDataTest folder
for subfolder_name in os.listdir(base_path):
    # create path for each subfolder
    folder_path = os.path.join(base_path, subfolder_name)

    # get the name of the meter from the subfolder name, make lowercase
    meter_name = subfolder_name.lower()
    #may need to replace " " with "_"
    df = combine_csvs(folder_path, meter_name)
    if not df.empty:
        all_data.append(df) 